# Data Literacy
#### University of Tübingen, Winter Term 2021/22
## Exercise Sheet 1
&copy; 2021 Prof. Dr. Philipp Hennig & Jonathan Wenger

This sheet is **due on Monday, October 25, 2021 at 12noon sharp (i.e. before the start of the lecture).**

---

## Data Manipulation and Visualization

This first coding exercise will introduce you to some basic concepts of working with data. As a use case we will consider statistics of the ongoing COVID-19 pandemic from two different sources. You will download the data, extract and pre-process the subset we are interested in for downstream analysis and generate some basic visualizations. These will also answer some initial questions you might have about the data and generate new questions regarding estimation of the latent number of infections, the spread of the virus and its mortality.

The tasks will primarily serve as an introduction to `pandas`, a Python tool for data manipulation and basic analysis and `matplotlib` a framework for plotting.

<img src="https://blog.dask.org/images/split-apply-combine.png" alt="split-apply-combine" style="width: 600px;"/>

### Initial Setup

For these homework exercises we expect basic familiarity with Python 3. If you are new to Python there are countless tutorials on the internet, e.g. https://realpython.com/python-first-steps/. As a reference for `pandas` functionality consult the [documentation](https://pandas.pydata.org/docs/).

**Task:** Install the packages `matplotlib`, `numpy` and `pandas` and make sure the cell below executes without errors.

In [2]:
# Make inline plots vector graphics
%matplotlib inline
from IPython.display import set_matplotlib_formats

set_matplotlib_formats("pdf", "svg")

# Plotting setup
import matplotlib.pyplot as plt
import datetime

# Package imports
import numpy as np
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

### German COVID-19 Data

We begin by downloading the COVID-19 data for Germany from the [Robert Koch Institute](https://www.rki.de/).

**Task:** Download the CSV data from the given URL using `pandas` and inspect a few rows from it.

In [32]:

# Link to current data of the RKI
url = "http://www.arcgis.com/sharing/rest/content/items/f10774f1c63e40168479a1feb6c7ca74/data"

# Read CSV data from URL
data_rki = pd.read_csv(url) 

# Inspect first few rows of the data
data_rki.head()

,FID,IdBundesland,Bundesland,Landkreis,Altersgruppe,Geschlecht,AnzahlFall,AnzahlTodesfall,Meldedatum,IdLandkreis,Datenstand,NeuerFall,NeuerTodesfall,Refdatum,NeuGenesen,AnzahlGenesen,IstErkrankungsbeginn,Altersgruppe2
0,1,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,2021/01/19 00:00:00,1001,"23.10.2021, 00:00 Uhr",0,-9,2021/01/11 00:00:00,0,1,1,Nicht übermittelt
1,2,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,2021/01/19 00:00:00,1001,"23.10.2021, 00:00 Uhr",0,-9,2021/01/16 00:00:00,0,1,1,Nicht übermittelt
2,3,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,5,0,2021/01/19 00:00:00,1001,"23.10.2021, 00:00 Uhr",0,-9,2021/01/19 00:00:00,0,5,0,Nicht übermittelt
3,4,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,2021/01/20 00:00:00,1001,"23.10.2021, 00:00 Uhr",0,-9,2021/01/15 00:00:00,0,1,1,Nicht übermittelt
4,5,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,2,0,2021/01/20 00:00:00,1001,"23.10.2021, 00:00 Uhr",0,-9,2021/01/16 00:00:00,0,2,1,Nicht übermittelt


**Task:** Based on the shortened description of the data below, compute the cumulative number of cases both in Germany and in Tübingen as of today. Check the RKI dashboard to verify your result.

> #### Data Description of the RKI Covid-19-Dashboard (https://corona.rki.de)
>
> The data has the following features:
> - ...
> - Landkreis: Name of the county
> - ...
> - AnzahlFall: Number of cases in the respective population group.
> - ...
> - NeuerFall:
>    - 0: Case is contained in the data of today and the previous day
>    - 1: Case is only contained in today's data
>    - -1: Case is only contained in the previous day's data

Source (in German): https://www.arcgis.com/home/item.html?id=f10774f1c63e40168479a1feb6c7ca74

In [33]:
# Compute the cumulative number of cases
mask = (data_rki['NeuerFall']==1)&(data_rki['Meldedatum']=='2021/10/22 00:00:00')
cases_germany = data_rki[mask].loc[:,'AnzahlFall'].sum() 
print(f"Total confirmed cases of COVID-19 in Germany: \t{cases_germany:,}")

# Compute the cumulative number of cases in Tübingen
mask = mask & (data_rki['Landkreis']=='LK Tübingen')
cases_tuebingen = data_rki[mask].loc[:,'AnzahlFall'].sum() 
print(f"Total confirmed cases of COVID-19 in Tübingen: \t{cases_tuebingen:,}")

Total confirmed cases of COVID-19 in Germany: 	11,051
Total confirmed cases of COVID-19 in Tübingen: 	27


### Incidence Trend

As we recently entered the second lockdown, the main concern over the coming days is whether, and how much the incidence rates are dropping again. 

**Task:** Create three plots of the difference between the number of new cases on any given day and the day directly preceding it normalized by the population size for Germany, Baden-Württemberg and Tübingen. Can you make out a trend already?

In [41]:
data_rki_sorted = data_rki.sort_values(by=['Meldedatum'], ascending=True)
reported_days = data_rki_sorted['Meldedatum'].unique()

In [67]:
def delta_cases(location_mask):
    
    deltas = []
    for date, previous_date in zip(reported_days[1:], reported_days[:-1]):
        print(date)
        current_date_mask = (data_rki['Meldedatum']== date)
        previous_date_mask = (data_rki['Meldedatum'] == previous_date)

        mask_new_cases = current_date_mask & location_mask & (data_rki['NeuerFall']==1)
        mask_corrected_cases = current_date_mask & location_mask & (data_rki['NeuerFall']==-1)
        mask_previous_cases = previous_date_mask & location_mask & (data_rki['NeuerFall']==1)
    
        current_cases = data_rki[mask_new_cases].loc[:,'AnzahlFall'].sum()
        previous_cases = data_rki[mask_previous_cases].loc[:,'AnzahlFall'].sum() 
        corrections = data_rki[mask_corrected_cases].loc[:,'AnzahlFall'].sum()
        corrected_cases = previous_cases - corrections
        
        delta = current_cases - corrected_cases
        deltas.append(delta)
        print(delta)
        print(current_cases)
        print(corrected_cases)
        print(previous_cases)
    return deltas


In [72]:
data_rki.iloc[-10:]

,FID,IdBundesland,Bundesland,Landkreis,Altersgruppe,Geschlecht,AnzahlFall,AnzahlTodesfall,Meldedatum,IdLandkreis,Datenstand,NeuerFall,NeuerTodesfall,Refdatum,NeuGenesen,AnzahlGenesen,IstErkrankungsbeginn,Altersgruppe2
2558585,2558586,16,Thüringen,LK Altenburger Land,A80+,W,1,0,2021/10/05 00:00:00,16077,"23.10.2021, 00:00 Uhr",0,-9,2021/10/05 00:00:00,-9,0,0,Nicht übermittelt
2558586,2558587,16,Thüringen,LK Altenburger Land,A80+,W,1,0,2021/10/07 00:00:00,16077,"23.10.2021, 00:00 Uhr",0,-9,2021/10/02 00:00:00,0,1,1,Nicht übermittelt
2558587,2558588,16,Thüringen,LK Altenburger Land,A80+,W,1,0,2021/10/12 00:00:00,16077,"23.10.2021, 00:00 Uhr",0,-9,2021/10/11 00:00:00,-9,0,1,Nicht übermittelt
2558588,2558589,16,Thüringen,LK Altenburger Land,A80+,W,1,0,2021/10/14 00:00:00,16077,"23.10.2021, 00:00 Uhr",0,-9,2021/10/14 00:00:00,-9,0,0,Nicht übermittelt
2558589,2558590,16,Thüringen,LK Altenburger Land,A80+,W,2,0,2021/10/15 00:00:00,16077,"23.10.2021, 00:00 Uhr",0,-9,2021/10/15 00:00:00,-9,0,0,Nicht übermittelt
2558590,2558591,16,Thüringen,LK Altenburger Land,A80+,W,2,0,2021/10/18 00:00:00,16077,"23.10.2021, 00:00 Uhr",0,-9,2021/10/18 00:00:00,-9,0,0,Nicht übermittelt
2558591,2558592,16,Thüringen,LK Altenburger Land,A80+,W,1,0,2021/10/20 00:00:00,16077,"23.10.2021, 00:00 Uhr",0,-9,2021/10/17 00:00:00,-9,0,1,Nicht übermittelt
2558592,2558593,16,Thüringen,LK Altenburger Land,A80+,W,2,0,2021/10/21 00:00:00,16077,"23.10.2021, 00:00 Uhr",1,-9,2021/10/21 00:00:00,-9,0,0,Nicht übermittelt
2558593,2558594,16,Thüringen,LK Altenburger Land,A80+,W,2,0,2021/10/21 00:00:00,16077,"23.10.2021, 00:00 Uhr",0,-9,2021/10/21 00:00:00,-9,0,0,Nicht übermittelt
2558594,2558595,16,Thüringen,LK Altenburger Land,unbekannt,W,1,0,2021/10/20 00:00:00,16077,"23.10.2021, 00:00 Uhr",0,-9,2021/10/20 00:00:00,-9,0,0,Nicht übermittelt


In [69]:
data_rki['Meldedatum']== '2020/01/23 00:00:00'

0          False
1          False
2          False
3          False
4          False
           ...  
2558590    False
2558591    False
2558592    False
2558593    False
2558594    False
Name: Meldedatum, Length: 2558595, dtype: bool

In [68]:
delta_cases(True)
#data_rki[current_date_mask & location_mask & (data_rki['NeuerFall']==1)]

2020/01/23 00:00:00
0
0
0
0
2020/01/28 00:00:00
0
0
0
0
2020/01/29 00:00:00
0
0
0
0
2020/01/31 00:00:00
0
0
0
0
2020/02/01 00:00:00
0
0
0
0
2020/02/03 00:00:00
0
0
0
0
2020/02/04 00:00:00
0
0
0
0
2020/02/05 00:00:00
0
0
0
0
2020/02/06 00:00:00
0
0
0
0
2020/02/07 00:00:00
0
0
0
0
2020/02/11 00:00:00
0
0
0
0
2020/02/17 00:00:00


KeyboardInterrupt: 

In [46]:
# Population sizes
population_sizes = {"Germany": 83783942,
                   "Baden-Württemberg": 11023424,
                   "Tübingen": 228678}


# Germany
location_mask = True
germany_deltas = delta_cases(location_mask) / population_sizes['Germany'] 

# Baden Württemberg
location_mask =(data_rki['Bundesland'] == 'Baden-Württemberg')
bawue_deltas = delta_cases(location_mask) / population_sizes["Baden-Württemberg"] 


# Tübingen
location_mask =(data_rki['Landkreis'] == 'LK Tübingen')
tuebingen_deltas = delta_cases(location_mask) / population_sizes["Tübingen"] 



KeyboardInterrupt: 

In [ ]:
import matplotlib.dates as mdates

# Plot timeseries
fig, axs = plt.subplots(1, 3, figsize=(10, 3.5), sharey=False)
ax[0].plot(germany_deltas)
ax[1].plot(bawue_deltas)
ax[2].plot(tuebingen_deltas)


### Estimating the Basic Reproduction Number $R_0$

Next, we will compute a rough estimate of the _basic reproduction number_ $R_0$ of COVID-19. The reproduction number is used to measure the transmission potential of a disease and constitutes the average number of infections caused by a typical case in a completely susceptible population.

[^1]: https://www.healthknowledge.org.uk/public-health-textbook/research-methods/1a-epidemiology/epidemic-theory

**Task:** Compute and plot a running estimate of the basic reproduction number by computing the ratio between the number of new infections in the last four days and the number of new infections in the four days before that. What can you say about the estimate of the reproduction rate for the current day and over time? If you had to make policy decisions based on this estimate, what would your response be?

*Hint:* To help illustrate your answer to the last two questions, you can use the `datetime` format and `matplotlib` to draw vertical lines on specific dates.

Source $R_0$ estimation: https://www.rki.de/DE/Content/Infekt/EpidBull/Archiv/2020/Ausgaben/17_20.pdf?__blob=publicationFile

In [ ]:
# Create dataframe of number of new cases
data_rki_cases = None # TODO

# Compute number of new cases in last four days and in four days prior to that

# Estimate the basic reproduction number R0

# Show data from the previous week


In [ ]:
# Plot reproduction rate


### Global COVID-19 Data

Next, we will take a look at the global case numbers for the corona virus from [OurWorldInData.org](https://ourworldindata.org/coronavirus-testing).

**Task:** Download the CSV data from the given URL and parse the `date` column into `datetime` format of the form `2021-11-01`. Why is it important to have a dedicated format for dates instead of just a string representation?

In [ ]:
# Link to world-wide COVID data
url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"

# Read CSV data from URL
data_owid = None

# Parse data

# Inspect dataframe and datatypes


**Task:** Subset the world-wide data with the countries of interest and create a new dataframe with seven columns, namely `iso_code`, `date`, `new_cases`, `new_cases_per_million`, `new_cases_smoothed_per_million`, a newly computed `case_fatality_rate` and the `positive_rate`.

*Hint:* The case fatality rate (CFR) is defined as the fraction of deaths and cases.

In [ ]:
# Countries of interest (CoIs)
cois = ["DEU", "GBR", "USA", "FRA", "NLD"]

# Subset worldwide data with CoIs

# Create CFR column

# Remove unnecessary columns


In [ ]:
data_owid.head()

### Pandemic Severity across Countries

**Task:** From the world-wide data generate timeseries plots showing new cases per million inhabitants (both raw and smoothed), the computed case fatality rate, and the rate of positive tests. Use log-axes where appropriate and make sure your plots are legible and have properly formatted dates! 
 
How do you explain the falling case fatality rate and the increasing positive rate over time? Which country fares the best in terms of the pandemic, which the worst?

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10, 3.5))

# Plotting colors
colors = dict(zip(cois, ["C0", "C1", "C2", "C3", "C4"]))

# New Cases:

# Case fatality rate

# Rate of positive tests

# Date formatting
